## Installed Required Packages

In [28]:
import sys
!{sys.executable} -m pip install fake-useragent

  Running setup.py bdist_wheel for fake-useragent ... done
  Stored in directory: /Users/jieyuwang/Library/Caches/pip/wheels/07/04/1d/bbd8ba7d692add504b44552504b7df239bddf56aa3387cee2b
Successfully built fake-useragent


In [235]:
!{sys.executable} -m pip install BeautifulSoup

    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/private/var/folders/pd/lj_g7cpj6nq6jqn0_47v2fxc0000gn/T/pip-build-6a69sbyc/BeautifulSoup/setup.py", line 22
        print "Unit tests have failed!"
                                      ^
    SyntaxError: Missing parentheses in call to 'print'. Did you mean print(int "Unit tests have failed!")?
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /private/var/folders/pd/lj_g7cpj6nq6jqn0_47v2fxc0000gn/T/pip-build-6a69sbyc/BeautifulSoup/


In [ ]:
!{sys.executable} -m pip install selenium

Download Chrome WebDriver: https://sites.google.com/a/chromium.org/chromedriver/getting-started

Then include the ChromeDriver location in your PATH environment variable

## Web Crapper Functions

In [1]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import re
def craw_one_page(url):
    user_agent = UserAgent()
    page = requests.get(url,headers={'user-agent':user_agent.chrome})
    content=page.content.decode('utf-8')
    soup = BeautifulSoup(content,'lxml')
    
    dict_result={}
    
    source=soup.find_all('div',attrs={'pane-header'})
    tabs = [div.string for div in source]
    tabs=tabs[:-2]
    
    
    source=soup.find_all('div',attrs={'text-content'})
    tabs_values = [div.string for div in source]
    
    for i in range(len(tabs)):
        dict_result[tabs[i]]=tabs_values[i]
    
    source=soup.find_all('a',attrs={'i'})
    keys = [div["title"] for div in source]
    
    source=soup.find_all('div',attrs={'s_val'})
    values = [div["title"] for div in source]
    
    for i in range(len(keys)):
        dict_result[keys[i]]=values[i]
    return dict_result

In [106]:
page = requests.get("https://search.wikileaks.org/plusd/?qproject[]=ps&qproject[]=cc&qproject[]=fp&qproject[]=ee&qproject[]=cg&q=China#result", headers={'user-agent':user_agent.chrome})
content=page.content.decode('utf-8')
soup = BeautifulSoup(content,'lxml')

In [112]:
Tagged_text=soup.find_all('a',target={'_blank'}, )
products = [div.string for div in Tagged_text]

In [47]:
import time 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
def crawl_search_source(search_url):
    browser=webdriver.Chrome()
    browser.get(search_url)
    try:
        ele=browser.find_element_by_id("button_next500")
    except:
        pass
    i=0
    while (i<500):
        try:
            ele.click()
        except:
            time.sleep(0.1)
            i=i+1
    return browser.page_source

In [48]:
def find_links_from_source(soup):
    Tagged_text=soup.find_all('a',target={'_blank'}, )
    set_url=set()
    for x in Tagged_text:
        url=x["href"]
        if (url[0:14]=="/plusd/cables/"):
            set_url.add(x["href"])
    return set_url

In [49]:
import json
def save_obj(data):
    with open('data_crawled.json', 'w') as fp:
        json.dump(data, fp)

In [50]:
def crawl_search_result(set_url):
    url_head= 'https://search.wikileaks.org'
    final_result={}
    i=0
    print ("total number of links: {}".format(len(set_url)))
    for x in set_url:
        print (x[14:-5], "{} out of {} jobs done".format(i+1, len(set_url)))
        final_result[x[14:]]=craw_one_page(url_head+x)
        if (i/2000==0):
            save_obj(final_result)
        i=i+1
    save_obj(final_result)

In [51]:
search_url="https://search.wikileaks.org/plusd/?qproject[]=ps&qproject[]=cg&qproject[]=cc&qproject[]=fp&qproject[]=ee&q=&qtfrom=1972-01-01&qtto=1972-12-31#result"

In [52]:
source=crawl_search_source(search_url)
soup = BeautifulSoup(source,'lxml')
set_url=find_links_from_source(soup)
crawl_search_result(set_url)

total number of links: 4
72TEHRAN1381_a 1 out of 4 jobs done
72TEHRAN5055_a 2 out of 4 jobs done
72TEHRAN1164_a 3 out of 4 jobs done
72TEHRAN4789_a 4 out of 4 jobs done


In [64]:
a=soup.find_all('p', style={"font-size:14px"})

In [68]:
for div in a:
    print (div)

<p style="font-size:14px">Total number of matching documents: 4. <a class="st" href="#st">Modify search</a></p>
